In [1]:
import torch
import torchtext
import numpy as np
from cnn.cnn import TEXT, model

In [2]:
TEXT.vocab.vectors

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.0361, -0.3528, -0.4494,  ..., -0.3391, -0.0521, -0.2626],
        [-0.8892,  0.3043,  0.9224,  ..., -0.2417, -0.1520,  0.0683],
        [ 0.2916,  0.0795, -0.0095,  ...,  0.3854,  0.3772, -1.5852]])

In [3]:
from torchtext import data
dataFields = {"comment_text": ("comment_text", TEXT)}

data_set = data.TabularDataset(path='./data/sampled_train/train.json', 
                                            format='json',
                                            fields=dataFields, 
                                            skip_header=False)

In [4]:
data_set.examples[1].comment_text

['click',
 'below',
 'for',
 'a',
 'free',
 'download',
 'of',
 'a',
 'colorfully',
 'illustrated',
 'n',
 'page',
 'e',
 'book',
 'on',
 'the',
 'zionist',
 'engineered',
 'intentional',
 'destruction',
 'of',
 'western',
 'civilization']

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_iterator = torchtext.data.Iterator(data_set, batch_size=64, device=device, 
                                     sort=False, sort_within_batch=False, 
                                     repeat=False,shuffle=False)

In [6]:
Preds=[]
with torch.no_grad():
    model.eval()
    for batch in data_iterator:

        torch.cuda.empty_cache()
    
        text = batch.comment_text    
        predictions = model(text).squeeze(1)         
        Preds+=[torch.sigmoid(predictions).detach().numpy()]
    
        torch.cuda.empty_cache()
Preds = (np.vstack(Preds)>0.5).astype(int)

In [7]:
import pandas as pd
df= pd.read_csv("./data/train_preprocessed.csv")

In [8]:
for i, col in enumerate(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]):
    df[col] = Preds[:, i]

In [9]:
df.head()

,Unnamed: 0,file_id,user_id,subforum_id,num_contexts,label,text,comment_text,label_int,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1,12834217_2,572066,1346,0,noHate,12834217_2,In order to help increase the booklets downloa...,0,0,0,0,0,0,0
1,3,12834217_4,572066,1346,0,hate,12834217_4,Click below for a FREE download of a colorfull...,1,0,0,0,0,0,0
2,10,12834493_1,572058,1396,0,hate,12834493_1,She may or may not be a Jew but she 's certain...,1,1,0,0,0,0,0
3,20,12849464_2,575975,1346,0,noHate,12849464_2,I think you should write a book as well,0,0,0,0,0,0,0
4,23,12849464_5,575975,1346,1,noHate,12849464_5,I guess I 'll just stick to homeschooling my k...,0,0,0,0,0,0,0


In [13]:
 df[['comment_text','label_int']+["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].sum(axis = 0)

label_int        957
toxic            144
severe_toxic       0
obscene           18
threat             0
insult            17
identity_hate      5
dtype: int64

In [11]:
len(df)

1914